In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import nltk #natural language toolkit

In [2]:
data=pd.read_excel('./dialog_talk_agent(correct) (1).xlsx')
data.head()

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,I can help you work smarter instead of harder
3,Describe yourself,I can help you work smarter instead of harder
4,tell me about yourself,I can help you work smarter instead of harder


In [3]:
data.dropna(inplace=True)

In [4]:
data.shape

(1587, 2)

In [5]:
import re
def text_normalization(text):
    text=text.lower()
    text=re.sub(r'[^a-z0-9]',' ',text)
    return text

In [6]:
data['doc_tp']=data['Context'].apply(text_normalization)
data.head()

,Context,Text Response,doc_tp
0,Tell me about your personality,Just think of me as the ace up your sleeve.,tell me about your personality
1,I want to know you better,I can help you work smarter instead of harder,i want to know you better
2,Define yourself,I can help you work smarter instead of harder,define yourself
3,Describe yourself,I can help you work smarter instead of harder,describe yourself
4,tell me about yourself,I can help you work smarter instead of harder,tell me about yourself


In [7]:
import nltk
nltk.download('wordnet')
from nltk.stem import wordnet
lema=wordnet.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahitha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
sent=data['doc_tp'][0]
sent

'tell me about your personality'

In [9]:
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
from nltk import pos_tag

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mahitha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Mahitha\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [10]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
def sent_lema(sentence):
    taglist=pos_tag(nltk.word_tokenize(sentence),tagset=None)
    lema_sent=[]
    for token,pos_token in taglist:
        if token in stop:
            continue
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lema.lemmatize(token,pos_val)
        lema_sent.append(lema_token)
    return " ".join(lema_sent) 

In [12]:
print(data['Context'][0])
print('CleanText:',sent)
print('Lemmatized clean text:',sent_lema(sent))

Tell me about your personality
CleanText: tell me about your personality
Lemmatized clean text: tell personality


In [13]:
data['lemma']=data['doc_tp'].apply(sent_lema)
data.head()

,Context,Text Response,doc_tp,lemma
0,Tell me about your personality,Just think of me as the ace up your sleeve.,tell me about your personality,tell personality
1,I want to know you better,I can help you work smarter instead of harder,i want to know you better,want know good
2,Define yourself,I can help you work smarter instead of harder,define yourself,define
3,Describe yourself,I can help you work smarter instead of harder,describe yourself,describe
4,tell me about yourself,I can help you work smarter instead of harder,tell me about yourself,tell


In [14]:
import pickle
pickle.dump(data,open('chat_lemmatization.pickle','wb'))

**Word embedding**

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
cv=CountVectorizer()

In [17]:
x=cv.fit_transform(data['lemma']).toarray()
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [18]:
pickle.dump(x,open('chat_matrix.pickle','wb'))

In [19]:
features=cv.get_feature_names()
print(features)

['21', 'abort', 'absolutely', 'abysmal', 'actually', 'adore', 'advice', 'advise', 'affirmative', 'afraid', 'afternoon', 'age', 'agree', 'ah', 'ahah', 'ahaha', 'ahahah', 'ahahaha', 'ahead', 'almost', 'alone', 'already', 'alright', 'alrighty', 'also', 'always', 'amaze', 'amazing', 'angry', 'annoy', 'annoying', 'annul', 'answer', 'anymore', 'anything', 'anytime', 'apologise', 'apologize', 'apology', 'apparently', 'appreciate', 'ask', 'asleep', 'assist', 'assistance', 'attractive', 'aways', 'awesome', 'awful', 'baby', 'back', 'bad', 'bear', 'beautiful', 'bed', 'beg', 'best', 'bestie', 'birth', 'birthday', 'bore', 'boring', 'bos', 'bot', 'brainy', 'bravo', 'brilliant', 'buddy', 'busy', 'bye', 'cancel', 'care', 'celebrate', 'certainly', 'chat', 'chatbot', 'cheer', 'childhood', 'city', 'clever', 'come', 'confirm', 'cook', 'cookie', 'cool', 'correct', 'could', 'country', 'course', 'crack', 'crazy', 'cry', 'cute', 'cuz', 'date', 'day', 'dear', 'define', 'definitely', 'depress', 'describe', 'dis

In [20]:
data_bow=pd.DataFrame(x,columns=features)
data_bow.head()

,21,abort,absolutely,abysmal,actually,adore,advice,advise,affirmative,afraid,...,yap,ye,yea,yeah,year,yeh,yep,yes,yet,yup
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
Q='i want to know about your personality'
Q

'i want to know about your personality'

In [22]:
#step1: cleaning
Q_clean=text_normalization(Q)
#step2: Lemmatization and remove stop words
Q_lema=sent_lema(Q_clean)
#step3: convert into bagofwords with same dimensions as your corpus
Q_bow=cv.transform([Q_lema]).toarray()
print(Q_lema)
print(Q_clean)
print(Q_bow)

want know personality
i want to know about your personality
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [23]:
x.shape

(1587, 419)

In [24]:
# cosine similarity
from sklearn.metrics import pairwise_distances

In [25]:
cos=1-pairwise_distances(x,Q_bow,metric='cosine')
ind=cos.argmax()
print('most similar index position {} and similarity value={}'.format(ind,cos[ind]))

most similar index position 12 and similarity value=[0.81649658]


In [26]:
data['Context'].loc[ind]

'I want to know more about you'

In [27]:
# jaccard index(intersection over union)
from sklearn.metrics import jaccard_similarity_score

In [28]:
Q_bow

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [29]:
Q_bow_repeat=Q_bow.repeat(5,axis=0)
Q_bow_repeat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

**TFIDF**

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [31]:
x_tfidf=tfidf.fit_transform(data['doc_tp']).toarray()
x_tfidf

array([[0.        , 0.        , 0.40628345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.64176014,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [32]:
cos=1-pairwise_distances(x,Q_bow,metric='cosine')
ind=cos.argmax()
print('most similar index position {} and similarity value={}'.format(ind,cos[ind]))

most similar index position 12 and similarity value=[0.81649658]


In [33]:
data['Context'].loc[ind]

'I want to know more about you'

In [36]:
while(True):
    Q=input('User: ')
    if Q=='Quit'or Q=='quit':
        print('Bye!!!')
        break
    #step-1: Cleaning
    Q_lema=text_normalization(Q)
    #step-2:Lemmatization and remove stop words
    #step-3:Convert into bag of word with samedimension as your corpus
    Q_tfidf=tfidf.transform([Q_lema]).toarray()
    cos=1-pairwise_distances(x,Q_bow,metric='cosine')
    ind=cos.argmax()
    if ind > 0.2:
        res=data.loc[data['Context']==data['Context'].loc[ind], 'Text Response']
        print('System:',res.iloc[0])
    else:
        print('Could you ask in another way!!!')
    

User: hai
System: I can help you work smarter instead of harder
User: quit
Bye!!!
